# Convert a RESQML file to Geoscience Objects

This example shows how to load a RESQML file and convert its elements into Geoscience Objects.

*Note:*
The data for this sample is not included in the repository and the following two files
will need to be downloaded to `samples/data-converters/python/convert-resqml/data`

- https://github.com/bp/resqpy/blob/master/example_data/block.epc
- https://github.com/bp/resqpy/blob/master/example_data/block.h5

Our first step is to define our Evo workspace metadata and provide a cache directory to store the generated files.

In [ ]:
import os

from evo.data_converters.common import EvoWorkspaceMetadata

org_id = ""
workspace_id = "9c86938d-a40f-491a-a3e2-e823ca53c9ae"

cache_root = os.path.join(os.getcwd(), "data/cache")
workspace_metadata = EvoWorkspaceMetadata(org_id=org_id, workspace_id=workspace_id, cache_root=cache_root)

### Conversion without memory threshold
The memory threshold can be specified in the conversion options passed in to the converter, and an example of this is given in the example following this example. If the threshold is not specified, as in this case, the upper default threshold of 8GB is in effect.

Now we can convert our RESQML file using the `convert_resqml` function.

_Note:_ Currently only the following RESQML objects are supported:
- IjkGridRepresentation converted to UnstructuredHexGrid
- Surface converted to TriangulatedMesh

_Note on CRS:_ The way the EPSG code to use for the Coordinate Reference System is determined is as follows: If there is a CRS associated with the Grid/Surface/etc to get the `epsg_code` then that is used. If not then the model CRS is used if present. If there is no model CRS, the default `epsg_code` passed in via the options is used.

The sample file does not have a coordinate reference system specified, so the default EPSG code (32650) passed in to the routine will be used.

In [ ]:
from evo.data_converters.resqml.importer import ResqmlConversionOptions, convert_resqml

file_name = os.path.join(os.getcwd(), "data/block.epc")

epsg_code = 32650

# Override the default option and export all cells, instead of just the active cells.
options = ResqmlConversionOptions(active_cells_only=False)

geoscience_objects = convert_resqml(
    file_name, evo_workspace_metadata=workspace_metadata, epsg_code=epsg_code, options=options
)

We can loop through the objects and inspect each one.

In [ ]:
for geoscience_object in geoscience_objects:
    print("-----------------------------------------------------------------------------------------------")
    print(geoscience_object.json_dumps(indent=4))

The file contains:
- The grid ROOT with 10 attributes
- The surface delaunay triangulated surface with no attributes


The grid ROOT has the following Attributes from the file:
- ACTIVE, by convention RESQML indicates active/inactive cells with a boolean array named ACTIVE
- NETGRS
- POROSITY
- PERMX
- PERMY
- PERMZ

And the following attributes that are generated by the conversion:
- ACTNUM, A boolean array indicating the active cells, only generated if active_cells_only is False
- I, the I cell coordinate
- J, the J cell coordinate
- K, the K cell coordinate

### Conversion with memory threshold
The memory threshold is an optional setting in the 'options' passed in to the converter. It is specified in bytes and will limit the conversion accordingly by simply truncating the conversion. The example below shows this in action with a very small threshold.

Note that when the memory_threshold is not specified, a maximum default threshold of 8GB is in effect.

In [ ]:
# Override the default option and export all cells, instead of just the active cells.
# We are also specifying the memory_threshold, in bytes.
options = ResqmlConversionOptions(active_cells_only=False, memory_threshold=1024)

geoscience_objects = convert_resqml(
    file_name, evo_workspace_metadata=workspace_metadata, epsg_code=epsg_code, options=options
)

Looping through the converted objects confirms the threshold caused truncation

In [ ]:
for geoscience_object in geoscience_objects:
    print("-----------------------------------------------------------------------------------------------")
    print(geoscience_object.json_dumps(indent=4))